In [66]:
import pandas as pd  # for data handling
import matplotlib.pyplot as plt  # for visualization
import re  # for regular expressions
plt.rc('figure', figsize=(10, 6))

#### Methods for Pre Processing

In [67]:

def load(f: str) -> pd.DataFrame:
    """
    Loads an Excel file and returns its contents as a Pandas DataFrame.
    
    Args:
        f (str): The name of the file to be loaded. It must be an Excel file with a valid extension (xlsx, xls, xlm).
    
    Returns:
        pd.DataFrame: DataFrame containing the contents of the Excel file.
    
    Raises:
        ValueError: If the file format is not supported or the file does not have an appropriate extension.
    """
    pattern = r'.*\.(xlsx|xls|xlsm|xlm)$'
    if re.match(pattern, f):
        df = pd.read_excel(f)
        return df
    else:
        raise ValueError("File format is not supported. Please provide a valid Excel file (xlsx, xls, xlm).")

In [68]:
def detail_report(df: pd.DataFrame):
    """
    Generates and prints a detailed report of the given Pandas DataFrame. 

    This includes:
    - Printing the number of rows and columns.
    - Displaying the first 10 rows of the DataFrame.
    - Displaying the last 10 rows of the DataFrame.
    - Showing the information (e.g., column types, non-null counts, etc.) of the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame for which the details need to be generated.
    """

    rows, cols = df.shape
    print('\n Rows - {0} and Columns - {1} \n'.format(rows, cols))
    print('--------- First 10 Rows ------------')
    display(df.head(10))  # first 10 rows of the data frame

    print('--------- Last 10 Rows -------------')
    display(df.tail(10))  # last 10 rows of the data frame

    print('--------- DataFrame Info ----------')
    display(df.info())  # information about the data frame

    print('---------- describe ---------')
    display(df.iloc[:, 1:].describe())


In [69]:
def replace_blank_with_zero(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces blank spaces, certain patterns, and NaN values with zero in a given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to be processed.

    Returns:
        pd.DataFrame: The cleaned DataFrame with blank spaces, '--', and NaN values replaced by 0.
    
    Steps:
        1. Compares the original DataFrame with the cleaned one.
        2. Displays rows that changed after the operation.
        3. Replaces the following patterns with 0.
        4. Displays the cleaned DataFrame.
    """

    # Create a copy of the original DataFrame
    original_df = df.copy()

    # Define a function for identifying blank spaces or '--'
    is_blank_or_invalid = lambda x: bool(re.match(r'^\s*$', str(x))) or x == '--'

    # Replace invalid values with 0
    cleaned_df = df.replace([r'^\s*$', '--'], 0, regex=True).fillna(0)

    # Identify and display rows that changed after the operation
    changed_rows = original_df[~original_df.eq(cleaned_df).all(axis=1)]
    print(' -------- Rows Changed (After Replacement) ------- ')
    display(changed_rows)

    print('----------- Cleaned Data Frame ------------')
    display(cleaned_df)
    return cleaned_df

In [70]:
def duplicate_rows(df: pd.DataFrame) -> pd.DataFrame:

    """
    Identifies and removes duplicate rows from the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to check for duplicates.

    Returns:
        pd.DataFrame: The DataFrame with duplicate rows removed.

    Steps:
        1. Count and display the number of duplicate rows.
        2. Remove duplicate rows from the DataFrame.
        3. Display the resulting DataFrame after duplicates are removed.
    """
    # Debug: Original data
    print("Original DataFrame (before duplicate removal):")
    display(df)

    # Identify duplicate rows across the entire DataFrame (not ignoring any columns)
    duplicate_counts = df.duplicated().sum()

    print(" -------- Number of Duplicated Rows ----------- ")
    print(duplicate_counts)

    # Remove duplicate rows
    df_cleaned = df[~df.duplicated()]

    # Debug: After duplicate removal
    print("DataFrame after removing duplicates:")
    display(df_cleaned)

    return df_cleaned



In [86]:
def conversion_str_to_int(df, start_col=1):
    """
    Converts all string values in the specified columns of a DataFrame to integers.

    This function processes all columns starting from the given index (start_col) 
    and converts string representations of numbers to integers by removing any 
    non-numeric characters, including commas.

    Args:
        df (pd.DataFrame): The DataFrame to process.
        start_col (int): The starting column index to apply the transformation. Defaults to 0.

    Returns:
        pd.DataFrame: The DataFrame with converted values.
    """
    def safe_conversion(x):
        # Remove non-digit characters
        cleaned = re.sub(r'[^0-9]', '', str(x))
        # Convert to integer if cleaned is not empty, else default to 0
        return int(cleaned) if cleaned else 0

    # Apply the conversion only to the specified columns
    for col in df.columns[start_col:]:
        df[col] = df[col].apply(safe_conversion)

    return df

In [89]:
def remove_outliers(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Removes rows containing outliers based on the IQR method and rows with zero immigration values 
    in any country of citizenship column.

    This function:
    1. Uses the IQR method to calculate outlier thresholds for numeric columns.
    2. Identifies rows with outliers or zero immigration values in any country of citizenship column.
    3. Displays the rows that are being removed.
    4. Returns both a cleaned DataFrame and a DataFrame containing the removed rows.

    Args:
        df (pd.DataFrame): The input DataFrame to process.

    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: The cleaned DataFrame with outliers and zero immigration rows removed.
            - pd.DataFrame: A DataFrame containing the rows with zero immigration.
    """
    # Calculate IQR for numeric columns
    
    print(" -------- Actual Data Frame --------- ")
    display(df)
    
    Q1 = df.iloc[:, 1:].quantile(0.25)
    Q3 = df.iloc[:, 1:].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify rows with outliers
    mask_outliers = ((df.iloc[:, 1:] < lower_bound) | (df.iloc[:, 1:] > upper_bound)).any(axis=1)

    # Identify rows with zero immigration in any "country of citizenship" column
    mask_zero_any_country = (df.iloc[:, 1:].filter(like='Citizenship of Country') == 0).any(axis=1)

    # Combine both conditions for rows to remove
    rows_to_remove = mask_outliers | mask_zero_any_country

    # Extract rows being removed into a new DataFrame
    removed_rows_df = df[rows_to_remove]

    # Display rows being removed
    print(" -------- Rows Removed (Outliers or Zero Immigration in Citizenship Columns) --------- ")
    display(removed_rows_df)

    # Remove rows from the original DataFrame
    df_cleaned = df[~rows_to_remove]
    
    print('----------- Cleaned Data Frame, After Removing Outliers ------------')
    display(df_cleaned)
    
    # Return cleaned DataFrame and DataFrame of removed rows
    return df_cleaned, removed_rows_df


In [73]:
def data_processing(f):
    """
    Process the data from the given file.

    This function loads data from an Excel file, cleans it by replacing 
    blank values and invalid entries with zeros, converts string 
    representations of numbers to integers, removes duplicate rows, 
    and generates a detailed report before and after the cleaning process.

    Args:
        f (str): The file path of the data to be processed.

    Returns:
        tuple: A tuple containing the original DataFrame and the cleaned DataFrame.
    """
    # Load the data from the given file
    df = load(f)

    # Generate and display a detailed report of the raw data
    print('--------- Raw Data Report ------------')
    detail_report(df)

    # Replace blank and invalid values with zeros
    cleaned_df = replace_blank_with_zero(df)

    # Convert string values to integers in numerical columns
    cleaned_df = conversion_str_to_int(cleaned_df)

    # Remove duplicate rows from the DataFrame
    # cleaned_df = duplicate_rows(cleaned_df)

    # Generate and display a detailed report for the cleaned data
    print('--------- Cleaned Data Report ------------')
    detail_report(cleaned_df)

    # Return both the original and cleaned DataFrames
    return cleaned_df


In [74]:
def save_toexcel(df: pd.DataFrame, file_name: str):
    """
    Saves a given DataFrame to an Excel file without adding the index.

    Args:
        df (pd.DataFrame): The DataFrame to be saved.
        file_name (str): The name (with path) of the Excel file to save the DataFrame to.
    
    Returns:
        None: Saves the file directly to the specified path.
    """
    df.to_excel(file_name, index=False)


In [75]:
def map_country_name_with_index(df, cleaned_df):
    """
    Maps the 'Citizenship of Country' column from the original DataFrame (df)
    to the corresponding indices in the cleaned DataFrame (cleaned_df).

    Args:
        df (pd.DataFrame): The original DataFrame containing the 'Citizenship of Country' column.
        cleaned_df (pd.DataFrame): The cleaned DataFrame where the mapping will be stored.

    Returns:
        pd.DataFrame: The cleaned DataFrame with a new column 'Mapped Country' containing
                      the mapped country names based on the original DataFrame.
    """
    # Ensure the column exists in the original DataFrame
    if 'Country of Citizenship' not in df.columns:
        raise KeyError("'Citizenship of Country' column is not present in the original DataFrame.")

    # Map the 'Country of Citizenship' column using proper .loc syntax
    try:
        for index in cleaned_df.index:
            # Map the 'Country of Citizenship' column using .loc
            cleaned_df.loc[index, 'Country of Citizenship'] = df.loc[index, 'Country of Citizenship']
    except KeyError as e:
        print("Error during mapping. Cleaned DataFrame indices may not match original DataFrame indices.")
        raise e


    print('----------- Cleaned Data Frame, After Mapping Country Names ------------')
    display(cleaned_df)

    return cleaned_df


In [82]:
def remove_rows_with_zero_immigration(df):
    """
    Removes rows from the DataFrame that contain zero in at least one column.

    Args:
        df (pd.DataFrame): The input DataFrame to process.

    Returns:
        pd.DataFrame: The DataFrame with rows containing zero removed.
    """
    # Filter rows where no column contains 0
    zero_mask = (df.iloc[:, 1:] == 0).any(axis=1)
    df_cleaned = df[~zero_mask]

    # df_cleaned = df[(df != 0).all(axis=1)]

    return df_cleaned


#### Education visa

In [84]:
def clean_and_save_study_visa_data(file_path: str):
    """
    Processes the study visa data from the given file path.

    This function performs the following steps:
    1. Loads the data using the data_processing function.
    2. Removes outliers and rows with zero immigration values using the remove_outliers function.
    3. Saves the cleaned DataFrame to a new Excel file.

    Args:
        file_path (str): The path of the file to process.

    Returns:
        None
    """
    # Load and clean the data from the specified file
    df = data_processing(file_path)

    display(df)
    # remove rows, if any exist with zero immigration
    df_cleaned = remove_rows_with_zero_immigration(df)

    # Remove outliers and rows with zero immigration values in specified columns
    df_cleaned, outlier_df = remove_outliers(df_cleaned)

    df_cleaned = map_country_name_with_index(df, df_cleaned)
    # Construct the cleaned file name by inserting 'cleaned_' before the actual file name
    cleaned_file_name = file_path.replace('.xlsx', '_cleaned.xlsx')

    # Save the cleaned DataFrame to the new Excel file
    save_toexcel(df_cleaned, cleaned_file_name)

countrywise

In [90]:
# Load and clean the data from the specified file
clean_and_save_study_visa_data('data/education visa/education_visa_countrywise.xlsx')

--------- Raw Data Report ------------

 Rows - 225 and Columns - 25 

--------- First 10 Rows ------------


,Country of Citizenship,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,255,210,135,65,45,45,30,40,55,...,130,110,130,115,100,120,115,90,185,195
1,Albania,190,175,160,120,85,70,70,90,120,...,145,175,220,275,365,530,490,435,495,755
2,Algeria,190,195,195,210,220,220,230,330,410,...,"1,250","1,060","1,155","1,400","2,030","3,620","4,205","5,195","7,480","13,335"
3,Andorra,--,--,--,0,0,0,--,--,--,...,10,5,--,--,--,5,--,10,10,--
4,Angola,310,340,290,170,145,40,25,20,65,...,190,95,115,85,75,135,55,50,85,110
5,Anguilla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,--,0,0,0,0
6,Antigua and Barbuda,85,85,90,90,95,90,100,85,90,...,95,110,130,155,180,180,150,130,140,140
7,Argentina,480,575,490,330,295,260,215,190,160,...,195,210,260,280,340,400,345,635,865,905
8,Armenia,20,15,20,25,30,30,25,20,15,...,40,30,45,35,35,40,30,45,75,100
9,Aruba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,--,--,--,10,10,10


--------- Last 10 Rows -------------


,Country of Citizenship,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
215,Venezuela,630,800,685,590,595,605,650,800,810,...,"2,250","1,980","1,925","1,675","1,240",875,690,625,815,940
216,Vietnam,615,825,"1,105","1,655","1,750","1,695","1,475","1,380","1,570",...,"4,720","4,850","7,435","13,900","20,140","21,480","18,815","16,155","16,045","17,175"
217,"Virgin Islands, British",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,--,--,--
218,Western Sahara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,--,--,0,0,0,0
219,Yemen,115,135,155,165,195,250,250,270,265,...,310,295,300,365,420,420,395,265,250,265
220,Zambia,65,100,110,120,150,200,235,270,290,...,250,235,235,230,230,230,215,210,255,315
221,Zimbabwe,120,330,430,290,230,220,255,305,305,...,525,630,890,980,960,880,755,825,"1,045","1,425"
222,Other Countries,"1,360",330,185,150,130,125,135,120,75,...,75,55,50,45,25,10,--,--,--,--
223,Country of Citizenship not stated,115,120,115,135,95,90,130,165,190,...,285,190,95,30,5,5,--,0,0,0
224,Total unique persons,"122,620","145,900","158,080","164,435","168,540","170,395","172,285","179,055","184,095",...,"330,090","352,305","410,545","490,745","566,935","637,780","527,195","616,585","804,370","1,040,985"


--------- DataFrame Info ----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Country of Citizenship  225 non-null    object
 1   2000                    225 non-null    object
 2   2001                    225 non-null    object
 3   2002                    225 non-null    object
 4   2003                    225 non-null    object
 5   2004                    225 non-null    object
 6   2005                    225 non-null    object
 7   2006                    225 non-null    object
 8   2007                    225 non-null    object
 9   2008                    225 non-null    object
 10  2009                    225 non-null    object
 11  2010                    225 non-null    object
 12  2011                    225 non-null    object
 13  2012                    225 non-null    object
 14  2013                  

...

,Country of Citizenship,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2,Algeria,190,195,195,210,220,220,230,330,410,...,1250,1060,1155,1400,2030,3620,4205,5195,7480,13335
10,Australia,1030,1210,1000,835,770,820,760,820,765,...,845,745,710,705,710,665,480,520,595,605
15,Bangladesh,600,920,1275,1560,1735,1715,1650,1550,1570,...,2565,2685,3060,4230,6405,8375,7735,9860,12250,15820
27,Brazil,1485,1615,1300,1085,1180,1345,1525,1945,2385,...,8855,7450,9310,11790,13745,14490,11005,11055,14415,15615
33,"Cameroon, Federal Republic of",415,520,510,535,520,495,530,580,585,...,1595,1790,1865,1840,2000,2500,2260,2550,4055,8095
39,"China, People's Republic of",12205,21985,31245,37400,39910,39980,39945,41085,43085,...,107810,117830,131050,139515,141715,140535,116150,104370,99470,102150
40,Colombia,1370,1745,2130,1715,1655,1300,925,975,1200,...,1485,1565,2005,2850,3995,5580,4805,8050,12400,15205
64,France,6475,6925,6425,6555,6680,6965,8145,9020,9400,...,18900,20205,20700,21830,22500,23845,18200,26380,27010,26980
70,Germany,2150,2305,2245,2135,2290,2450,2590,2950,3145,...,2510,2455,2650,2730,2905,2945,1465,4985,4050,3005
71,Ghana,630,700,690,640,570,515,435,465,465,...,1095,1370,1510,1535,1555,2050,1870,1840,3455,9235


----------- Cleaned Data Frame, After Removing Outliers ------------


,Country of Citizenship,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,255,210,135,65,45,45,30,40,55,...,130,110,130,115,100,120,115,90,185,195
1,Albania,190,175,160,120,85,70,70,90,120,...,145,175,220,275,365,530,490,435,495,755
4,Angola,310,340,290,170,145,40,25,20,65,...,190,95,115,85,75,135,55,50,85,110
6,Antigua and Barbuda,85,85,90,90,95,90,100,85,90,...,95,110,130,155,180,180,150,130,140,140
7,Argentina,480,575,490,330,295,260,215,190,160,...,195,210,260,280,340,400,345,635,865,905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Uruguay,35,55,70,55,50,40,35,30,30,...,20,15,20,40,50,70,65,75,65,65
213,Uzbekistan,20,25,15,20,30,30,30,35,35,...,45,50,50,55,50,65,55,70,115,280
219,Yemen,115,135,155,165,195,250,250,270,265,...,310,295,300,365,420,420,395,265,250,265
220,Zambia,65,100,110,120,150,200,235,270,290,...,250,235,235,230,230,230,215,210,255,315


----------- Cleaned Data Frame, After Mapping Country Names ------------


,Country of Citizenship,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,255,210,135,65,45,45,30,40,55,...,130,110,130,115,100,120,115,90,185,195
1,Albania,190,175,160,120,85,70,70,90,120,...,145,175,220,275,365,530,490,435,495,755
4,Angola,310,340,290,170,145,40,25,20,65,...,190,95,115,85,75,135,55,50,85,110
6,Antigua and Barbuda,85,85,90,90,95,90,100,85,90,...,95,110,130,155,180,180,150,130,140,140
7,Argentina,480,575,490,330,295,260,215,190,160,...,195,210,260,280,340,400,345,635,865,905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Uruguay,35,55,70,55,50,40,35,30,30,...,20,15,20,40,50,70,65,75,65,65
213,Uzbekistan,20,25,15,20,30,30,30,35,35,...,45,50,50,55,50,65,55,70,115,280
219,Yemen,115,135,155,165,195,250,250,270,265,...,310,295,300,365,420,420,395,265,250,265
220,Zambia,65,100,110,120,150,200,235,270,290,...,250,235,235,230,230,230,215,210,255,315


provincewise

In [79]:
# Call the function
# clean_and_save_study_visa_data('data/education visa/education_visa_provincewise.xlsx')